In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [19]:
!pip install dagshub mlflow --quiet
!pip install imbalanced-learn==0.11.0 --quiet

In [20]:

import dagshub
import mlflow

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import xgboost as xgb
from category_encoders import WOEEncoder
import warnings
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline  # ✅ Corrected import
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import shap
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score



In [3]:
# dagshub.init(repo_owner='nkikn21', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=185ccf87-138d-400d-8943-2ae617d36110&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=0d62cdb8a17528916cdbe4fb90bfb58ee2eea20bccfaf8376303ce0bdd6f4a72




Output()

Accessing as nkikn21

Initialized MLflow to track repo "nkikn21/IEEE-CIS-Fraud-Detection"

Repository nkikn21/IEEE-CIS-Fraud-Detection initialized!

In [21]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "nkikn21"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "90ec7031365aea1b6ca271f4236c194e530973c8"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow")



In [22]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [23]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
# test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
# test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [8]:
# train_transaction.shape, train_identity.shape, test_transaction.shape, test_identity.shape

((590540, 394), (144233, 41), (506691, 393), (141907, 41))

In [36]:
# train_transaction.head()

invalid value encountered in greater
invalid value encountered in less
invalid value encountered in greater


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [37]:
# train_identity.head()

invalid value encountered in greater
invalid value encountered in less
invalid value encountered in greater


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [12]:
# test_identity.columns = test_identity.columns.str.replace('-', '_')

# print("Updated Test Columns:", test_identity.columns)

Updated Test Columns: Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')


In [24]:
train_joined = train_transaction.merge(train_identity, on="TransactionID", how="left")
# test_joined = test_transaction.merge(test_identity, on="TransactionID", how="left")

**Train-Test Split**

In [25]:
train_joined['TransactionDay'] = train_joined['TransactionDT'] // (24*60*60)
# test_joined['TransactionDay'] = test_joined['TransactionDT'] // (24*60*60)
cutoff_day = train_joined['TransactionDay'].max() - 30  # last 30 days for validation

train_data = train_joined[train_joined['TransactionDay'] <= cutoff_day]
val_data = train_joined[train_joined['TransactionDay'] > cutoff_day]

X_train = train_data.drop(columns=['isFraud', 'TransactionID'])
y_train = train_data['isFraud']

X_val = val_data.drop(columns=['isFraud', 'TransactionID'])
y_val = val_data['isFraud']

# X_test = test_joined.drop(columns=['TransactionID', 'isFraud'], errors='ignore')  # Ignore errors in case 'isFraud' isn't in test_joined


In [ ]:
# train_joined.head(10)

In [40]:
# train_joined.shape

(590540, 435)

In [ ]:
# X_train.shape, X_val.shape

In [26]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

In [27]:
s = X_train[cat_cols].nunique()

threshold = 3

woe_columns = list(s[s > threshold].index)
one_hot_columns = list(s[s <= threshold].index)

In [28]:
print(len(one_hot_columns), len(woe_columns))

21 10


In [29]:
len(num_cols)

402

# Cleaning

In [30]:
import mlflow
from sklearn.base import BaseEstimator, TransformerMixin

class Cleaning(BaseEstimator, TransformerMixin):
    
    def __init__(self, woe_columns, one_hot_columns, num_cols, log_mlflow=False, null_threshold=0.8):
        self.woe_columns = woe_columns
        self.one_hot_columns = one_hot_columns
        self.num_cols = num_cols
        self.log_mlflow = log_mlflow
        self.null_threshold = null_threshold

    def fit(self, X, y=None):
        # Drop columns with more than threshold% missing values
        null_frac = X.isnull().mean()
        self.to_drop = null_frac[null_frac > self.null_threshold].index.tolist()

        # Fill strategies for remaining columns
        self.woe_columns_fill_na = X[self.woe_columns].mode().T[0].to_dict()
        self.one_hot_columns_fill_na = X[self.one_hot_columns].mode().T[0].to_dict()
        self.num_cols_fill_na = X[self.num_cols].median().to_dict()

        # MLflow logging
        if self.log_mlflow:
            experiment_name = 'LR_Training'
            run_name = 'LR_Cleaning'
            
            mlflow.set_experiment(experiment_name)
            mlflow.start_run(run_name=run_name)

            mlflow.log_param("cat_cols_handling", "mode")
            mlflow.log_param("num_cols_handling", "median")
            mlflow.log_param("dropped_cols_threshold", self.null_threshold)
            mlflow.log_param("dropped_columns", self.to_drop)

            mlflow.end_run()
        
        return self

    def transform(self, X):
        X_transformed = X.copy()

        # Drop columns identified during fit
        X_transformed = X_transformed.drop(columns=self.to_drop, errors='ignore')

        # Fill WOE columns
        for col in self.woe_columns:
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.woe_columns_fill_na.get(col))

        # Fill one-hot columns
        for col in self.one_hot_columns:
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.one_hot_columns_fill_na.get(col))

        # Fill numeric columns
        for col in self.num_cols:
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.num_cols_fill_na.get(col))

        return X_transformed


# Feature Engineering

In [54]:
# from sklearn.base import BaseEstimator, TransformerMixin
# import pandas as pd
# import mlflow

# class FeatureEngineering(BaseEstimator, TransformerMixin):
    
#     def __init__(self, woe_columns=None, one_hot_columns=None, woe_mappings=None, woe_columns_fill_na=None, log_mlflow=False):
#         self.woe_columns = woe_columns
#         self.one_hot_columns = one_hot_columns
#         self.woe_mappings = woe_mappings
#         self.woe_columns_fill_na = woe_columns_fill_na
#         self.log_mlflow = log_mlflow

#     def fit(self, X, y=None):
#         # Assign default dicts if None (safe to do here)
#         self.woe_mappings_ = self.woe_mappings or {}
#         self.woe_columns_fill_na_ = self.woe_columns_fill_na or {}

#         # Optional: make sure columns exist
#         self.woe_columns_ = [col for col in self.woe_columns if col in X.columns]
#         self.one_hot_columns_ = [col for col in self.one_hot_columns if col in X.columns]

#         # Log to MLflow
#         if self.log_mlflow:
#             experiment_name = "LR_Training"
#             run_name = "LR_Feature_Engineering"
#             mlflow.set_experiment(experiment_name)
#             with mlflow.start_run(run_name=run_name):
#                 mlflow.log_param("woe_columns", self.woe_columns_)
#                 mlflow.log_param("one_hot_columns", self.one_hot_columns_)

#         return self

#     def transform(self, X):
#         X_transformed = X.copy()

#         # WOE encoding
#         for col in self.woe_columns_:
#             mapping = self.woe_mappings_.get(col, {})
#             default_val = mapping.get(self.woe_columns_fill_na_.get(col), 0)
#             new_col = f'{col}_woe'
#             X_transformed[new_col] = X_transformed[col].map(mapping).fillna(default_val)
#             X_transformed.drop(columns=col, inplace=True)

#         # One-hot encoding
#         X_transformed = pd.get_dummies(
#             X_transformed, 
#             columns=self.one_hot_columns_, 
#             drop_first=True, 
#             dummy_na=True,
#             dtype=int
#         )

#         return X_transformed


In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import mlflow

class FeatureEngineering(BaseEstimator, TransformerMixin):
    
    def __init__(self, woe_columns=None, one_hot_columns=None, woe_mappings=None, woe_columns_fill_na=None, log_mlflow=False):
        self.woe_columns = woe_columns
        self.one_hot_columns = one_hot_columns
        self.woe_mappings = woe_mappings
        self.woe_columns_fill_na = woe_columns_fill_na
        self.log_mlflow = log_mlflow

    def fit(self, X, y=None):
        # Assign default dicts if None (safe to do here)
        self.woe_mappings_ = self.woe_mappings or {}
        self.woe_columns_fill_na_ = self.woe_columns_fill_na or {}

        # Optional: make sure columns exist
        self.woe_columns_ = [col for col in self.woe_columns if col in X.columns]
        self.one_hot_columns_ = [col for col in self.one_hot_columns if col in X.columns]

        # Log to MLflow
        if self.log_mlflow:
            experiment_name = "LR_Training"
            run_name = "LR_Feature_Engineering"
            mlflow.set_experiment(experiment_name)
            with mlflow.start_run(run_name=run_name):
                mlflow.log_param("woe_columns", self.woe_columns_)
                mlflow.log_param("one_hot_columns", self.one_hot_columns_)

        return self

    def transform(self, X):
        X_transformed = X.copy()

        # WOE encoding
        for col in self.woe_columns_:
            mapping = self.woe_mappings_.get(col, {})
            default_val = mapping.get(self.woe_columns_fill_na_.get(col), 0)
            new_col = f'{col}_woe'
            X_transformed[new_col] = X_transformed[col].map(mapping).fillna(default_val)
            X_transformed.drop(columns=col, inplace=True)

        # One-hot encoding
        X_transformed = pd.get_dummies(
            X_transformed, 
            columns=self.one_hot_columns_, 
            drop_first=True, 
            dummy_na=True,
            dtype=int
        )

        return X_transformed


# Feature Selection

In [55]:
# class FeatureSelection(BaseEstimator, TransformerMixin):
#     def __init__(self, model, correlation_threshold=0.8, n_features_to_select=20, log_mlflow=False):
#         self.model = model
#         self.correlation_threshold = correlation_threshold
#         self.n_features_to_select = n_features_to_select
#         self.log_mlflow = log_mlflow
    
#     def fit(self, X, y=None):
#         # Step 1: Remove highly correlated features
#         corr_matrix = X.corr().abs()
#         upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#         self.to_drop = [column for column in upper.columns if any(upper[column] > self.correlation_threshold)]
    
#         # Step 2: Run RFE to select top N features
#         self.rfe = RFE(self.model, n_features_to_select=self.n_features_to_select)
#         self.rfe.fit(X, y)
    
#         all_rfe_features = X.columns[self.rfe.support_].tolist()
    
#         # Step 3: Final selected features = RFE features minus highly correlated
#         self.selected_features = [col for col in all_rfe_features if col not in self.to_drop]
    
#         # Step 4: Log to MLflow if enabled
#         if self.log_mlflow:
#             # MLflow logging code would go here
#             experiment_name = 'LR_Training'
#             run_name = 'LR_Feature_Selection'
            
#             mlflow.set_experiment(experiment_name)
#             mlflow.start_run(run_name=run_name)
            
#             mlflow.log_param("RFE_all_features", all_rfe_features)
#             mlflow.log_param("Highly_correlated_dropped", self.to_drop)
#             mlflow.log_param("Selected_features", self.selected_features)
#             mlflow.end_run()
#             pass
    
#         return self
    
#     def transform(self, X):
#         return X[self.selected_features]

In [32]:
class FeatureSelection(BaseEstimator, TransformerMixin):
    def __init__(self, model, correlation_threshold=0.8, n_features_to_select=20, log_mlflow=False):
        self.model = model
        self.correlation_threshold = correlation_threshold
        self.n_features_to_select = n_features_to_select
        self.log_mlflow = log_mlflow

    def fit(self, X, y=None):
        # Step 1: Remove highly correlated features
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.to_drop = [column for column in upper.columns if any(upper[column] > self.correlation_threshold)]
    
        # Step 2: Run RFE to select top N features
        self.rfe = RFE(self.model, n_features_to_select=self.n_features_to_select)
        self.rfe.fit(X, y)
    
        all_rfe_features = X.columns[self.rfe.support_].tolist()
    
        # Step 3: Final selected features = RFE features minus highly correlated
        self.selected_features = [col for col in all_rfe_features if col not in self.to_drop]
    
        # Step 4: Log to MLflow if enabled
        if self.log_mlflow:
            experiment_name = 'LR_Training'
            run_name = 'LR_Feature_Selection'
            
            mlflow.set_experiment(experiment_name)
            mlflow.start_run(run_name=run_name)
            
            mlflow.log_param("RFE_all_features", all_rfe_features)
            mlflow.log_param("Highly_correlated_dropped", self.to_drop)
            mlflow.log_param("Selected_features", self.selected_features)

            mlflow.end_run()
    
        return self


    def transform(self, X):
        return X[self.selected_features]


# Training

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler

# Fast logistic regression training pipeline
imb_pipeline = ImbPipeline(steps=[
    ('undersampler', RandomUnderSampler(random_state=42, sampling_strategy=0.4)),
    
    ('cleaning', Cleaning(woe_columns=woe_columns, 
                          one_hot_columns=one_hot_columns, 
                          num_cols=num_cols, log_mlflow=True)),
    
    ('feature_engineering', FeatureEngineering(woe_columns=woe_columns, 
                                               one_hot_columns=one_hot_columns, log_mlflow=True)),
    
    ('feature_selection', FeatureSelection(
        model=LogisticRegression(
            solver='liblinear',
            penalty='l2',
            max_iter=300,
            random_state=42
        ),
        n_features_to_select=20,
        log_mlflow=True
    )),
    
    ('scaler', StandardScaler()),
    
    ('classifier', LogisticRegression(
        solver='liblinear',
        penalty='l2',
        max_iter=300,
        random_state=42
    ))
])


In [ ]:
imb_pipeline.fit(X_train, y_train)

🏃 View run LR_Cleaning at: https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/4/runs/21ef193fcc304fde8e7a2a6cf38d3993
🧪 View experiment at: https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/4
🏃 View run LR_Feature_Engineering at: https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/4/runs/d628386cc13b46a09ffe964a13c096c6
🧪 View experiment at: https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/4


invalid value encountered in greater
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.


In [ ]:
from sklearn.feature_selection import RFE


experiment_name = 'Logistic_Regression'
run_name = 'Pipeline_run'

mlflow.set_experiment(experiment_name)
mlflow.start_run(run_name=run_name)

mlflow.sklearn.log_model(imb_pipeline, "pipeline")

# Train the model
# imb_pipeline.fit(X_train, y_train)

# Log metrics (e.g., AUC, F1-score, etc.)
y_pred = imb_pipeline.predict(X_val)
y_pred_proba = imb_pipeline.predict_proba(X_val)[:, 1]

auc_score = roc_auc_score(y_val, y_pred_proba)
f1_score_val = f1_score(y_val, y_pred)
recall_score_val = recall_score(y_val, y_pred)

mlflow.log_metric("AUC", auc_score)
mlflow.log_metric("F1_Score", f1_score_val)
mlflow.log_metric("Recall", recall_score_val)

# Log model parameters
mlflow.log_param("RandomUnderSampler_Sampling_Strategy", 0.4)
mlflow.log_param("Classifier", "LogisticRegression")

# Optionally log additional parameters related to the model (e.g., hyperparameters for LogisticRegression)
mlflow.log_param("LogisticRegression_Max_Iter", 1000)
mlflow.log_param("LogisticRegression_Solver", "lbfgs")

mlflow.end_run()

In [35]:
# param_grid = {
#     'classifier__C': [0.01, 0.1, 1, 10],             # Regularization strength (for LogisticRegression)
#     'classifier__solver': ['lbfgs', 'liblinear'],     # Solvers (for LogisticRegression)
#     'classifier__penalty': ['l2', 'l1'],                   # Penalty type (for LogisticRegression)
#     # 'classifier__max_depth': [3, 5, 7],              # Max depth for XGBoost
#     # 'classifier__learning_rate': [0.01, 0.1, 0.2]    # Learning rate for XGBoost
# }

In [36]:
# from sklearn.model_selection import GridSearchCV  # Import GridSearchCV

# grid_search = GridSearchCV(
#     estimator=imb_pipeline,
#     param_grid=param_grid,
#     scoring='roc_auc',  # Use AUC as the scoring metric
#     cv=5,               # 5-fold cross-validation
#     n_jobs=-1,          # Use all CPU cores
#     verbose=1           # Print progress
# )

In [37]:
# grid_search.fit(X_train, y_train)
# best_model = grid_search.best_estimator_


RuntimeError: Cannot clone object FeatureEngineering(one_hot_columns=['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7',
                                    'M8', 'M9', 'id_12', 'id_15', 'id_16',
                                    'id_23', 'id_27', 'id_28', 'id_29', 'id_35',
                                    'id_36', 'id_37', 'id_38', 'DeviceType'],
                   woe_columns=['ProductCD', 'card4', 'card6', 'P_emaildomain',
                                'R_emaildomain', 'id_30', 'id_31', 'id_33',
                                'id_34', 'DeviceInfo'],
                   woe_columns_fill_na={}, woe_mappings={}), as the constructor either does not set or modifies parameter woe_columns_fill_na

In [ ]:
# print("Best hyperparameters:", grid_search.best_params_)
# print("Best AUC score during CV:", grid_search.best_score_)

# # 5. Evaluate the best model on the test set
# y_val_pred_proba = best_model.predict_proba(X_val)[:, 1]
# auc_test = roc_auc_score(y_val, y_val_pred_proba)
# print(f"Val AUC: {auc_val:.4f}")

In [ ]:
# from sklearn.metrics import f1_score, recall_score, precision_score

# # Predict binary labels (not probabilities)
# y_val_pred_labels = pipeline.predict(X_val)

# # F1 Score
# f1 = f1_score(y_val, y_val_pred_labels)
# print(f"Validation F1 Score: {f1:.4f}")

# # Recall
# recall = recall_score(y_val, y_val_pred_labels)
# print(f"Validation Recall: {recall:.4f}")

# # (Optional) Precision
# precision = precision_score(y_val, y_val_pred_labels)
# print(f"Validation Precision: {precision:.4f}")


In [ ]:
#  6. SHAP value analysis
# explainer = shap.Explainer(best_model, X_train)  # Use training data for SHAP explanation
# shap_values = explainer(X_test)  # Generate SHAP values for the test set

# # 7. Plot SHAP values (for feature importance)
# shap.summary_plot(shap_values, X_test)